In [1]:
%pwd

'/home/jingw222/myDirPy/MLPractices/Instacart'

In [2]:
%matplotlib inline
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.options.display.max_rows=200

In [4]:
dtype = {'order_id': np.int32, 'product_id': np.uint16, 'add_to_cart_order': np.uint8, 'reordered': np.int8,
        'user_id': np.int32, 'eval_set': np.int8, 'order_number': np.int8, 'order_dow': np.int8, 
         'order_hour_of_day': np.int8, 'days_since_prior_order': np.float32}

In [5]:
ord_pri_test = pd.read_csv('./data/stats/ord_pri_test.csv', dtype=dtype)

products = pd.read_csv('./data/stats/products.csv', index_col='product_id')

orders_test_eval1 = pd.read_csv('./data/stats/orders_test_eval1.csv')

orders_test = pd.read_csv('./data/stats/orders_test.csv', dtype=dtype)

In [9]:
ord_pri_test

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,cumsum_days_since_prior_order
4741003,1374495,9387,1,0,3,1,1,3,0.0
4741004,1374495,17668,2,0,3,1,1,3,0.0
4741005,1374495,15143,3,0,3,1,1,3,0.0
4741006,1374495,16797,4,0,3,1,1,3,0.0
4741007,1374495,39190,5,0,3,1,1,3,0.0
4741008,1374495,47766,6,0,3,1,1,3,0.0
4741009,1374495,21903,7,0,3,1,1,3,0.0
4741010,1374495,39922,8,0,3,1,1,3,0.0
4741011,1374495,24810,9,0,3,1,1,3,0.0
4741012,1374495,32402,10,0,3,1,1,3,0.0


In [8]:
ord_pri_test.sort_values(['user_id', 'order_number', 'add_to_cart_order'], inplace=True)

In [10]:
pred_prior_days = orders_test.loc[orders_test['eval_set']==1, ['user_id', 'days_since_prior_order']].set_index('user_id')

In [11]:
grouped = ord_pri_test.groupby(['user_id', 'product_id'])
grouped_ = ord_pri_test.groupby(['user_id'])

In [12]:
tot_prod_count_by_user = grouped_['product_id'].count().rename('tot_prod_count_by_user')
prod_count_by_user = grouped['product_id'].count().rename('prod_count_by_user')
prod_pct_by_user = (prod_count_by_user / tot_prod_count_by_user).rename('prod_pct_by_user').astype(np.float32).round(3)

cart_order_by_user = grouped['add_to_cart_order'].mean().rename('cart_order_by_user').astype(np.float32).round(3)

In [13]:
last_order_num_user = grouped_['order_number'].max().rename('last_order_num_user')
last_order_num_prod = grouped['order_number'].max().rename('last_order_num_prod')

last_order_num_by_prod_user = (pd.merge(last_order_num_prod.reset_index(), last_order_num_user.reset_index(), on='user_id')
                               .set_index(['user_id', 'product_id']))

In [14]:
isin_last_order = grouped.apply(lambda g: int(last_order_num_user[g.name[0]] in set(g['order_number']))).rename('isin_last_order')
isin_sed_last_order = grouped.apply(lambda g: int(last_order_num_user[g.name[0]]-1 in set(g['order_number']))).rename('isin_sed_last_order')
isin_thd_last_order = grouped.apply(lambda g: int(last_order_num_user[g.name[0]]-2 in set(g['order_number']))).rename('isin_thd_last_order')
isin_fth_last_order = grouped.apply(lambda g: int(last_order_num_user[g.name[0]]-3 in set(g['order_number']))).rename('isin_fth_last_order')

In [15]:
buy_hour_by_prod = grouped['order_hour_of_day'].apply(lambda g: g.mode().iloc[0]).rename('buy_hour_by_prod')
buy_dow_by_prod = grouped['order_dow'].apply(lambda g: g.mode().iloc[0]).rename('buy_dow_by_prod')

In [18]:
orders_test_eval1

,order_id,pred_order_dow,pred_order_hour_of_day
user_id,,,
3,2774568,5,3
4,329954,3,3
6,1528013,3,3
11,1376945,6,1
12,1356845,1,2
15,2161313,1,1
16,1416320,0,3
19,1735923,6,3
20,1980631,1,1


In [17]:
orders_test_eval1 = orders_test_eval1.set_index('user_id').drop('order_number', axis=1)
orders_test_eval1.rename(columns = {'order_dow': 'pred_order_dow', 'order_hour_of_day': 'pred_order_hour_of_day'}, inplace=True)

In [19]:
buy_hour_by_prod = buy_hour_by_prod.to_frame().join(orders_test_eval1['pred_order_hour_of_day'])
buy_hour_by_prod['buy_hour_equal_pred_hour'] = (buy_hour_by_prod['buy_hour_by_prod'] == buy_hour_by_prod['pred_order_hour_of_day']) * 1

buy_dow_by_prod = buy_dow_by_prod.to_frame().join(orders_test_eval1['pred_order_dow'])
buy_dow_by_prod['buy_dow_equal_pred_dow'] = (buy_dow_by_prod['buy_dow_by_prod'] == buy_dow_by_prod['pred_order_dow']) * 1

In [20]:
avg_prior_days_by_prod = grouped['cumsum_days_since_prior_order'].apply(lambda g: (g.max()-g.min())/(g.size-1))
avg_prior_days_by_user = avg_prior_days_by_prod.mean(level=0)
avg_prior_days_by_prod = avg_prior_days_by_prod.combine_first(avg_prior_days_by_prod.add(avg_prior_days_by_user, level=0, fill_value=0)).rename('avg_prior_days_by_prod')

/home/jingw222/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [21]:
avg_prior_days_by_prod

user_id  product_id
3        248            28.179094
         1005           28.179094
         1819           13.000000
         7503           28.179094
         8021           28.179094
         9387           19.000000
         12845          28.179094
         14992           7.000000
         15143          28.179094
         16797          45.000000
         16965          12.000000
         17668          29.500000
         18370          28.179094
         18599          19.000000
         21137          28.179094
         21903          19.000000
         22035          10.500000
         23650         124.000000
         24010          26.000000
         24810          66.500000
         28373          32.000000
         32402          53.500000
         38596          28.179094
         39190          14.777778
         39922          28.179094
         40604          28.179094
         42265          28.179094
         42557          28.179094
         43961          14.0

In [22]:
last_cumsum_prior_days_by_user_prod = grouped['cumsum_days_since_prior_order'].max()
last_cumsum_prior_days_by_user = grouped_['cumsum_days_since_prior_order'].max()

days_since_last_buy_prod = (last_cumsum_prior_days_by_user - last_cumsum_prior_days_by_user_prod + pred_prior_days.squeeze()).rename('days_since_last_buy_prod')

diff_days_avg_since = (days_since_last_buy_prod - avg_prior_days_by_prod).round(2).rename('diff_days_avg_since')

In [23]:
last_order_num_by_prod_user['last_order_num_prod_diff1'] = last_order_num_by_prod_user['last_order_num_user'] - last_order_num_by_prod_user['last_order_num_prod']
last_order_num_by_prod_user['last_order_num_prod_diff2'] = (last_order_num_by_prod_user['last_order_num_prod'] / last_order_num_by_prod_user['last_order_num_user']).round(2)

last_order_num_by_prod_user.drop(['last_order_num_prod', 'last_order_num_user'], axis=1, inplace=True)

In [24]:
last_order_num_by_prod_user

last_order_num_prod_diff1  last_order_num_prod_diff2
user_id product_id                                                      
3       248                                10                       0.17
        1005                                2                       0.83
        1819                                5                       0.58
        7503                                9                       0.25
        8021                               10                       0.17
        9387                                5                       0.58
        12845                               8                       0.33
        14992                               5                       0.58
        15143                              11                       0.08
        16797                               3                       0.75
        16965                               7                       0.42
        17668                               1                       0.92
        18370                               7                       0.42
        18599                               0                       1.00
        21137                              10                       0.17
        21903                               0                       1.00
        22035                               4                       0.67
        23650                               0                       1.00
        24010                               5                       0.58
        24810                               0                       1.00
        28373                               7                       0.42
        32402                               2                       0.83
        38596                              10                       0.17
        39190                               0                       1.00
        39922                              11                       0.08
        40604                              10                       0.17
        42265                               7                       0.42
        42557                               4                       0.67
        43961                               1                       0.92
        44683                               6                       0.50
        47766                               0                       1.00
        48523                               3                       0.75
        49683                               9                       0.25
4       1200                                2                       0.60
        2707                                4                       0.20
        7160                                3                       0.40
        7350                                4                       0.20
        11865                               2                       0.60
        17769                               2                       0.60
        19057                               2                       0.60
        21573                               0                       1.00
        22199                               1                       0.80
        25146                               1                       0.80
        25623                               0                       1.00
        26576                               0                       1.00
        35469                               2                       0.60
        36606                               4                       0.20
        37646                               2                       0.60
        42329                               3                       0.40
        43704                               2                       0.60
6       8424                                0                       1.00
        10644                               1                       0.67
        11068                               1                       0.67
     

In [25]:
merged = pd.concat((prod_pct_by_user, cart_order_by_user, 
                    isin_last_order, isin_sed_last_order, isin_thd_last_order, isin_fth_last_order, 
                    buy_hour_by_prod, buy_dow_by_prod,
                    diff_days_avg_since,
                    last_order_num_by_prod_user), axis=1)

In [26]:
merged = merged.join(products)

In [27]:
merged.head()

prod_pct_by_user  cart_order_by_user  isin_last_order  \
user_id product_id                                                          
3       248                    0.011               3.000                0   
        1005                   0.011               5.000                0   
        1819                   0.034               2.667                0   
        7503                   0.011               6.000                0   
        8021                   0.011               5.000                0   

                    isin_sed_last_order  isin_thd_last_order  \
user_id product_id                                             
3       248                           0                    0   
        1005                          0                    1   
        1819                          0                    0   
        7503                          0                    0   
        8021                          0                    0   

                    isin_fth_last_order  buy_hour_by_prod  \
user_id product_id                                          
3       248                           0                 3   
        1005                          0                 3   
        1819                          0                 3   
        7503                          0                 3   
        8021                          0                 3   

                    pred_order_hour_of_day  buy_hour_equal_pred_hour  \
user_id product_id                                                     
3       248                              3                         1   
        1005                             3                         1   
        1819                             3                         1   
        7503                             3                         1   
        8021                             3                         1   

                    buy_dow_by_prod  pred_order_dow  buy_dow_equal_pred_dow  \
user_id product_id                                                            
3       248                       3               5                       0   
        1005                      3               5                       0   
        1819                      0               5                       0   
        7503                      3               5                       0   
        8021                      3               5                       0   

                    diff_days_avg_since  last_order_num_prod_diff1  \
user_id product_id                                                   
3       248                      106.82                         10   
        1005                       8.82                          2   
        1819                      55.00                          5   
        7503                      85.82                          9   
        8021                     106.82                         10   

                    last_order_num_prod_diff2  aisle_id  department_id  
user_id product_id                                                      
3       248                              0.17       117             19  
        1005                             0.83        94              7  
        1819                             0.58        88             13  
        7503                             0.25       117             19  
        8021                             0.17        54             17

In [28]:
merged.to_csv('./data/test_merged_with_index.csv')

In [29]:
merged.reset_index(drop=True, inplace=True)

In [30]:
merged.to_csv('./data/test_merged_without_index.csv', index=False)